In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

data = pd.merge(ratings, movies, on='movieId')

user_item_matrix = data.pivot_table(index='userId', columns='title', values='rating', fill_value=0)

movie_similarity = cosine_similarity(user_item_matrix.T)

def get_top_n_recommendations(movie_title, n=5):
    movie_idx = list(user_item_matrix.columns).index(movie_title)
    similar_scores = movie_similarity[movie_idx]
    similar_movies = list(user_item_matrix.columns[np.argsort(similar_scores)[::-1]])[1:n+1]
    
    return similar_movies

movie_title_to_recommend = 'Avengers, The (2012)'
recommendations = get_top_n_recommendations(movie_title_to_recommend)

print(f"Top 5 recommendations for {movie_title_to_recommend}:")
for i, movie_title in enumerate(recommendations, start=1):
    print(f"{i}. {movie_title}")


Top 5 recommendations for Avengers, The (2012):
1. Guardians of the Galaxy (2014)
2. Iron Man 2 (2010)
3. Iron Man (2008)
4. X-Men: First Class (2011)
5. Captain America: The Winter Soldier (2014)


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

data = pd.merge(ratings, movies, on='movieId')

user_item_matrix = data.pivot_table(index='userId', columns='title', values='rating', fill_value=0)

movie_similarity_ratings = cosine_similarity(user_item_matrix.T)

genre_matrix = pd.get_dummies(movies['genres'])
movie_features_matrix = pd.concat([movies[['movieId']], genre_matrix], axis=1).set_index('movieId')

scaler = MinMaxScaler()
scaled_genre_matrix = scaler.fit_transform(movie_features_matrix)

num_movies = min(movie_similarity_ratings.shape[0], scaled_genre_matrix.shape[0])
movie_similarity_ratings = movie_similarity_ratings[:num_movies, :num_movies]
scaled_genre_matrix = scaled_genre_matrix[:num_movies, :]

movie_similarity_genres = cosine_similarity(scaled_genre_matrix)

alpha = 0.5
movie_similarity_combined = alpha * movie_similarity_ratings + (1 - alpha) * movie_similarity_genres

def get_top_n_recommendations(movie_title, n=5):
    movie_idx = list(user_item_matrix.columns).index(movie_title)
    similar_scores = movie_similarity_combined[movie_idx]
    similar_movies = list(user_item_matrix.columns[np.argsort(similar_scores)[::-1]])[1:n+1]
    
    return similar_movies

movie_title_to_recommend = 'Avengers, The (2012)'
recommendations = get_top_n_recommendations(movie_title_to_recommend)

print(f"Top 5 recommendations for {movie_title_to_recommend}:")
for i, movie_title in enumerate(recommendations, start=1):
    print(f"{i}. {movie_title}")


Top 5 recommendations for Avengers, The (2012):
1. Guardians of the Galaxy (2014)
2. Iron Man 2 (2010)
3. Iron Man (2008)
4. X-Men: First Class (2011)
5. Captain America: The Winter Soldier (2014)
